In [1]:
import json
with open('train/train.jsonl',encoding="utf-8") as f:
    train = [json.loads(jline) for jline in f.read().splitlines()]
len(train)

701

In [32]:
def item_to_string(item):
    s=''
    for para in item['original_doc']['_source']['body']:
        s+=para['text']+'\n'
    return s

In [2]:
max_length=0
for item in train[:30]:
    for para in item['original_doc']['_source']['body']:
        max_length=max(max_length,len(para['text'].split()))
print(max_length)

270


In [146]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

# Input 
text = "Diego Costa"

# To perform word (and sentence) segmentation
sentences = rdrsegmenter.tokenize(text) 
print(sentences)

[['Diego_Costa']]


In [43]:
para_cnt=0
for idx,item in enumerate(train):
    text=item_to_string(item)
    if '_' in text:
        print(idx,item['train_id'],text)
para_cnt

479 22994451 Ngay ở phút thứ 2, tân binh đắt giá Lacazette đã lập tức đặt dấu ấn của mình bằng pha đánh đầu hiểm hóc ở tỉ số cho Arsenal.
Tuy nhiên, đội chủ nhà liên tiếp để Leicester vươn lên dẫn trước 2-1 nhờ các bàn thắng của Okazaki (5') và Vardy ở phút 29.
[Video tổng hợp trận đấu ở cuối bài]
Arsenal và Leicester đã tạo nên một màn rượt đuổi điên rồ
Rất may là trước giờ nghỉ giải lao, Welbeck đã dễ dàng giúp Pháo thủ quân binh tỉ số 2-2. Nếu để thua trước khi hiệp 1 khép lại, tâm lý các học trò HLV Wenger sẽ bị đè nặng và chưa biết điều gì sẽ xảy ra.
Nhưng ngay cả như vậy, đội chủ sân Emirates cũng bước vào hiệp 2 với phong thái như "đãi khách". Phần vì hàng thủ của Arsenal lỏng lẻo, phần vì Leicester tấn công quá hay, mà Vardy đã có cú đúp cho riêng mình phút 56, đưa đội khách vươn lên dẫn 3-2.
Sự kịch tính và phấn khích được đẩy lên đỉnh điểm, khi liên tiếp trong các phút 83 và 85, Aaron Ramsey và tiền đạo vào sân thay người Giroud đã liên tiếp ghi những bàn thắng quý hơn vàng, 

0

In [46]:
item['original_doc']['_source']['body'][0]

{'content': 'Bàn thắng', 'text': 'Bàn thắng', 'type': 'text'}

In [101]:
# max_length=0
l=[]
for item in train:
    for para in item['original_doc']['_source']['body']:
        l.append(len(tokenize(para['text'])))

        # max_length=max(max_length,len(tokenize(para['text'])))
# print(max_length)

In [111]:
def match_arr(sub, arr, idx):
    for i in range(len(sub)):
        if sub[i]!=arr[idx+i]:
            return False
    return True

def find_array(sub,arr):
    if len(sub)==0 or len(arr)==0:
        return -1
    for i in range(len(arr)-len(sub)+1):
        if match_arr(sub,arr,i):
            return i
    return -1

def find_all_array(sub,arr):
    result=[]
    for i in range(len(arr)-len(sub)+1):
        if match_arr(sub,arr,i):
            result.append(i)
    return result


In [155]:
import itertools
from bs4 import BeautifulSoup

def tokenize(text):
    token_sentences = rdrsegmenter.tokenize(text)
    return list(itertools.chain.from_iterable(token_sentences))

def generate_ner_annotation(item,idx_item):
    result=[]

    for idx,html in enumerate(item["html_annotation"]):
        para_tokens = tokenize(item['original_doc']['_source']['body'][idx]['text'])
        tags = ['O']*len(para_tokens)

        soup = BeautifulSoup(html)
        events = soup.find_all("span", {"class": "tag"})
        for e in events:
            event_tokens = tokenize(e.text)
            event_pos = find_array(event_tokens,para_tokens)
            if event_pos==-1:
                print(event_tokens,para_tokens)
                print(idx_item, idx,'---------------------------')
                return []
                # print('--',event_tokens)
                # print('++',para_tokens)

            event_info_list = ref_id_to_info(e['event_id'],item['match_summary'])
            # print(event_info_list)
            for info in event_info_list:
                for attr in info[1]:
                    if attr!='ref_event_ids' and info[1][attr].strip()!='':
                        attr_tokens = tokenize(info[1][attr])
                        attr_pos_list = find_all_array(attr_tokens,event_tokens)
                        # if len(attr_pos_list)==0:
                            # print('---',info[1][attr])
                        for attr_pos in attr_pos_list:
                            tags[event_pos+attr_pos]='B'+'-'+info[0]+'_'+attr
                            for i in range(1,len(attr_tokens)):
                                tags[event_pos+attr_pos+i]='I'+'-'+info[0]+'_'+attr
                            # print(info[1][attr],tags[event_pos+attr_pos])

        # print(tags)
        result.append(list(zip(para_tokens,tags)))
    return result

        


        # events_info = ref_id_to_info(e['event_id'],item['match_summary'])
        # for info in events_info:
        #     if info[0]=='score_list':
                
    # for e in events:
    #     print("event_type:", e["data"])
    #     print("event_id:", e["event_id"])
    #     print("text:", e.text)

In [160]:
train[24]

{'train_id': '21275320',
 'original_doc': {'_id': '21275320',
  '_source': {'body': [{'content': 'HLV Phan Thanh Hùng phát biểu sau trận mở màn V-League 2017',
     'text': 'HLV Phan Thanh Hùng phát biểu sau trận mở màn V-League 2017',
     'type': 'text'},
    {'content': '<strong>[VIDEO]: HÀ NỘI FC 3-2 THAN QUẢNG NINH</strong>',
     'text': '[VIDEO]: HÀ NỘI FC 3-2 THAN QUẢNG NINH',
     'type': 'text'},
    {'content': '“Các bạn xem trận đấu sẽ biết ngay trọng tài đúng hay sai. Tôi không cần nói gì nhiều, chúng ta sẽ biết trọng tài xử trong trận hôm nay đúng hay chưa”, HLV Phan Thanh Hùng trả lời câu hỏi của phóng viên về công tác trọng tài chính điều khiển trận đấu ngày hôm nay, đặc biệt ở tình huống đáng nhẽ Than Quảng Ninh được hưởng một quả phạt đền phút 89.',
     'text': '“Các bạn xem trận đấu sẽ biết ngay trọng tài đúng hay sai. Tôi không cần nói gì nhiều, chúng ta sẽ biết trọng tài xử trong trận hôm nay đúng hay chưa”, HLV Phan Thanh Hùng trả lời câu hỏi của phóng viên về cô

In [161]:
train[24]['original_doc']['_source']['body']

{'content': 'Là cuộc tái ngộ của hai đội bóng từng tranh Siêu Cúp quốc gia những ngày cuối tháng 12.2016, cả Than Quảng Ninh và Hà Nội T&amp;T đều nhập cuộc máu lửa và sớm cho khán giả thấy được tầm vóc của hai đội bóng “ông tám lạng, bà nửa cân”.',
 'text': 'Là cuộc tái ngộ của hai đội bóng từng tranh Siêu Cúp quốc gia những ngày cuối tháng 12.2016, cả Than Quảng Ninh và Hà Nội T&amp;T đều nhập cuộc máu lửa và sớm cho khán giả thấy được tầm vóc của hai đội bóng “ông tám lạng, bà nửa cân”.',
 'type': 'text'}

In [158]:
soup = BeautifulSoup(train[24]['html_annotation'][9])
events = soup.find_all("span", {"class": "tag"})
for e in events:
    print(e.text)

Là cuộc tái ngộ của hai đội bóng từng tranh Siêu Cúp quốc gia những ngày cuối tháng 12.2016, cả Than Quảng Ninh và Hà Nội T&T đều nhập cuộc máu lửa và sớm cho khán giả thấy được tầm vóc của hai đội bóng “ông tám lạng, bà nửa cân”


In [156]:
generate_ner_annotation(train[24],24)

['Là', 'cuộc', 'tái_ngộ', 'của', 'hai', 'đội', 'bóng', 'từng', 'tranh', 'Siêu_Cúp', 'quốc_gia', 'những', 'ngày', 'cuối', 'tháng', '12.2016', ',', 'cả', 'Than_Quảng_Ninh', 'và', 'Hà_Nội', 'T&T', 'đều', 'nhập_cuộc', 'máu_lửa', 'và', 'sớm', 'cho', 'khán_giả', 'thấy', 'được', 'tầm_vóc', 'của', 'hai', 'đội', 'bóng', '“', 'ông', 'tám', 'lạng', ',', 'bà', 'nửa', 'cân', '”'] ['Là', 'cuộc', 'tái_ngộ', 'của', 'hai', 'đội', 'bóng', 'từng', 'tranh', 'Siêu_Cúp', 'quốc_gia', 'những', 'ngày', 'cuối', 'tháng', '12.2016', ',', 'cả', 'Than_Quảng_Ninh', 'và', 'Hà_Nội', 'T', '&', 'amp', ';', 'T', 'đều', 'nhập_cuộc', 'máu_lửa', 'và', 'sớm', 'cho', 'khán_giả', 'thấy', 'được', 'tầm_vóc', 'của', 'hai', 'đội', 'bóng', '“', 'ông', 'tám', 'lạng', ',', 'bà', 'nửa', 'cân', '”', '.']
24 9 ---------------------------


[]

In [152]:
import codecs
with codecs.open('train.txt', 'w', "utf-8") as f:
    # json.dump(doc, modified,ensure_ascii=False,indent=4)
    for idx,item in enumerate(train[:100]):
        try:
            ner_ann = generate_ner_annotation(item,idx)
        except:
            print('***********',idx)
        if len(ner_ann)==0:
            break
        for para in ner_ann:
            for token in para:
                f.write('{} {}\n'.format(token[0],token[1]))
            f.write('\n')

24 9 ---------------------------


In [120]:
len(['Tiền_đạo', 'Wilson', 'không', 'mắc', 'bất_cứ', 'sai_lầm', 'nào', 'để', 'nâng', 'tỷ_số', 'lên', '2-0', 'cho', 'Bournemouth', '.'])#, 'Tiền_đạo', 'Wilson', 'không', 'mắc', 'bất_cứ', 'sai_lầm', 'nào', 'để', 'nâng', 'tỷ_số', 'lên', '2-0', 'cho', 'Bournemouth', '.'] )

15

In [35]:
print(item_to_string(train[0]))

Video tổng hợp trận đấu:
Chelsea cần chiến thắng trước Stoke City ở Stamford Bridge để cân bằng kỷ lục 13 chiến thắng của Arsenal ở cuối mùa 2001/02. Cuối cùng, đoàn quân của HLV Conte đã hoàn thành nhiệm vụ này khi vượt qua đối thủ với tỷ số 4-2.
Ở trận đấu này, Stoke City đã cố gắng hết sức. Họ đã đặt Chelsea vào thế rượt đuổi tỷ số. Nhưng rồi, với đẳng cấp, The Blues đã vượt lên ở thời điểm quyết định.
Chelsea tiếp tục vô đối
Với chiến thắng này, Chelsea chính thức là ông Vua lượt đi Premier League với 49 điểm sau 19 vòng đấu. Người ta vẫn chưa tìm đối thủ đủ sức cắt đứt mạch chiến thắng của The Blues ở thời điểm này.
Bước vào trận đấu, Chelsea đón chào sự trở lại của Diego Costa. Cầu thủ người Tây Ban Nha sát cánh cùng Hazard và Willian trên hàng công. Với quá nhiều lợi thế trong tay, Chelsea đã gây sức ép lên khung thành đối thủ ngay sau tiếng còi khai cuộc.
Phút 18, Diego Costa đã có cơ hội cực kỳ thuận lợi để ghi bàn sau đường chuyền của Fabregas. Đáng tiếc, cú dứt điểm của tron

In [25]:
def ref_id_to_info(ref_id,match_summary):
    output=[]
    for category in match_summary:
        if category in {'score_list','card_list','substitution_list'}:
            if isinstance(match_summary[category],list):
                output+=[(category,item) for item in match_summary[category] if str(ref_id) in item['ref_event_ids']]
            else:
                if ref_id in match_summary[category]['ref_event_ids']:
                    output.append((category,match_summary[category]))
    return output


In [18]:
type([1,2,3])

list

In [24]:
ref_id_to_info(6,item['match_summary'])

[('score_list',
  {'player_name': 'Diego Costa',
   'time': '85',
   'team': 'Chelsea',
   'ref_event_ids': '6,7'})]

In [10]:
for

{'players': {'team1': 'Chelsea', 'team2': 'Stoke', 'ref_event_ids': '1'},
 'score_board': {'score1': '4', 'score2': '2', 'ref_event_ids': '1,6'},
 'score_list': [{'player_name': 'Cahill',
   'time': '34',
   'team': 'Chelsea',
   'ref_event_ids': '2,7'},
  {'player_name': 'Willian',
   'time': '57',
   'team': 'Chelsea',
   'ref_event_ids': '3,7'},
  {'player_name': 'Crouch',
   'time': '64',
   'team': 'Stoke',
   'ref_event_ids': '4,7,8'},
  {'player_name': 'Diego Costa',
   'time': '85',
   'team': 'Chelsea',
   'ref_event_ids': '6,7'},
  {'player_name': 'Willian',
   'time': '65',
   'team': 'Chelsea',
   'ref_event_ids': '5,7'},
  {'player_name': 'Martins Indi',
   'time': '46',
   'team': 'Stoke',
   'ref_event_ids': '7'}],
 'card_list': [],
 'substitution_list': []}